In [1]:
from ldapkg.mymodel import LDA_OPT
from collections import Counter
import numpy as np

In [6]:
M = 20 # Document numbers
V = 220
k = 3
N_d = np.random.randint(15, 20, M)
vocabulary = list(range(V))

np.random.seed(12345)
# Generate alpha
alpha_known = np.array([0.15, 0.35, 0.5])

# Generate beta
beta = np.random.uniform(0, 0.5, size = (V, k))
for i in range(k):
    for j in range(V):
        if j < (i+1)/k*V and j>=i/k*V:
           beta[j,i] += 0.5
beta_known = beta/np.sum(beta, 0)

# Generate theta
theta = np.random.dirichlet(alpha_known, size = M)

# Generate document
doc = []
for m in range(M):
    doc_single = []
    for n in range(N_d[m]):
        z = np.random.multinomial(1, theta[m]).reshape((k,1)) #choose a topic
        prob = beta_known @ z
        value = np.random.multinomial(1,  prob[:,0])
        word = np.where(value==1)[0][0]
        doc_single.append(word)
    doc.append(doc_single)



doc_model = []

# Generate corpus
for m in range(M):
    cnt = Counter()
    for word in doc[m]:
        cnt[word] += 1
    doc_model.append(dict(cnt))

In [7]:
# Implement LDA model
lda_model = LDA_OPT(3, 1500, 500, 500)
alpha_model, beta_model = lda_model.fit(doc_model, vocabulary)

- Show the resuls

In [8]:
alpha_model

array([ 0.49252064,  0.34149931,  0.16598005])

In [9]:
# Average difference in beta
beta_model[:,[0,1,2]] = beta_model[:,[2,1,0]] ## Rearrange the column
np.mean(np.abs(beta_model-beta_known))

0.005975641752468671